In [1]:
from dotenv import load_dotenv
import os
import requests
from julep import Julep
import yaml
import time

load_dotenv()

True

In [2]:
def experiences_to_str(experiences: list[dict]) -> str:
    """
    Returns, in reverse order, work experience stringified.
    """
    res = []
    for experience in reversed(experiences):
        company = experience.get('company')
        title = experience.get('title')
        starts_at = experience.get('starts_at')
        ends_at = experience.get('ends_at', 'current')
        description = experience.get('description')

        experience_str = (
            f"Company: {company}\n"
            f"Title: {title}\n"
            f"Start Date: {starts_at}\n"
            f"End Date: {ends_at}\n"
            f"Description: {description}\n"
        )
        res.append(experience_str)

    return '\n'.join(res)

In [3]:
def education_to_str(education: list[dict]) -> str:
    res = []
    for edu in reversed(education):
        field_of_study = edu.get('field_of_study')
        degree_name = edu.get('degree_name')
        school = edu.get('school')
        description = edu.get('description')
        grade = edu.get('grade')
        activities = edu.get('activities_and_societies')
        starts_at = edu.get('starts_at')
        ends_at = edu.get('ends_at')

        edu_string = (
            f"School: {school}\n"
            f"Degree: {degree_name} in {field_of_study}\n"
            f"Grade: {grade}\n"
            f"Activities: {activities}\n"
            f"Description: {description}\n"
            f"Start Date: {starts_at}\n"
            f"End Date: {ends_at}\n"
        )
        res.append(edu_string)

    return '\n'.join(res)

In [4]:
def person_to_str(person: dict):
    # Basic Information
    first_name: str | None = person.get('first_name')
    last_name: str | None = person.get('last_name')
    gender: str | None = person.get('gender')
    birth_date: str | None = person.get('birth_date')

    # Location Information
    city: str | None = person.get('city')
    state: str | None = person.get('state')
    country: str | None = person.get('country_full_name')

    # Profile Headline and Summary
    headline: str | None = person.get('headline')  # tagline
    summary: str | None = person.get('summary')  # longer headline

    # Professional Information
    industry: str | None = person.get('industry')
    occupation: str | None = person.get('occupation')

    # Experiences and Education
    experiences: list | None = person.get('experiences')
    experiences_str: str = experiences_to_str(experiences) if experiences else ""
    education: list | None = person.get('education')
    education_str: str = education_to_str(education) if education else ""

    return (
        f"Name: {first_name or ''} {last_name or ''}\n"
        f"Gender: {gender or 'N/A'}\n"
        f"Birth Date: {birth_date or 'N/A'}\n"
        f"Location: {city or ''}, {state or ''}, {country or ''}\n"
        f"Headline: {headline or 'N/A'}\n"
        f"Summary: {summary or 'N/A'}\n"
        f"Industry: {industry or 'N/A'}\n"
        f"Occupation: {occupation or 'N/A'}\n"
        f"Experiences:\n{experiences_str or 'No experience listed'}\n"
        f"Education:\n{education_str or 'No education listed'}"
    )

In [7]:
headers = {'Authorization': f"Bearer {os.getenv('PROXYCURL_API_KEY')}"}
url = "https://nubela.co/proxycurl/api/v2/linkedin"
params = {'linkedin_profile_url': 'https://www.linkedin.com/in/kent-g-00ba0743/'}
person: dict = requests.get(url, headers=headers, params=params).json()
person_str = person_to_str(person)
print(person_str)

Name: Kent G.
Gender: N/A
Birth Date: N/A
Location: Queens, New York, United States
Headline: Founder & CEO at Abstract Operators
Summary: playing with robot corporations
Industry: N/A
Occupation: Founder at Abstract Operators
Experiences:
Company: Kwiat Quantum Information Group, University of Illinois at Urbana-Champaign
Title: Research Assistant
Start Date: {'day': 1, 'month': 9, 'year': 2014}
End Date: {'day': 30, 'month': 11, 'year': 2015}
Description: Part Time

Company: Fannie Mae
Title: Financial Engineer I
Start Date: {'day': 1, 'month': 6, 'year': 2017}
End Date: {'day': 30, 'month': 6, 'year': 2018}
Description: Develop algorithms and analyses for the pricing and policies of mortgage backed securities.

Company: Fannie Mae
Title: Financial Economist I
Start Date: {'day': 1, 'month': 6, 'year': 2018}
End Date: {'day': 30, 'month': 4, 'year': 2019}
Description: •	Model Credit Risk Transfer (CRT) transactions in Python, accounting for deal terms such as maturity, delinquency an

In [8]:
julep_api_key = os.getenv('JULEP_OPENAI_API_KEY')
client = Julep(api_key=julep_api_key)

In [9]:
profile_creator = client.agents.create(
    name='Hero Story Writer',
    model='gpt-4o-mini',
    about='You are a professional writer who crafts engaging profiles that introduce individuals at the start of their hero’s journey. Highlight their potential, core traits, and hint at upcoming challenges. Your tone is inspirational, grand, yet relatable, connecting the reader to the hero’s beginnings and setting the stage for their transformation.',
)

In [22]:
with open('hero_writer.yaml', 'r') as f:
    hero_writer_yaml = f.read()

In [23]:
task = client.tasks.create(agent_id=profile_creator.id, **yaml.safe_load(hero_writer_yaml))

In [24]:
execution = client.executions.create(task_id=task.id, input={'person_linkedin_str': person_str})

In [26]:
while (result := client.executions.get(execution.id)).status not in ['succeeded', 'failed']:
    time.sleep(1)

if result.status == "succeeded":
    res = result.output
else:
    raise Exception(result.error)

In [34]:
choices = res.get('choices')
message = choices[0]['message']['content']
print(message)

```yaml
name: "Kent G."
background: "Hailing from the vibrant borough of Queens, New York, Kent G. stands at the precipice of innovation, where technology meets creativity. As the Founder and CEO of Abstract Operators, Kent is poised to revolutionize the way we interact with artificial intelligence through open-source orchestration and advanced Kubernetes solutions. With a foundation built on rigorous academic training and a diverse professional journey, Kent embodies the spirit of a true pioneer."
traits: 
    - "Visionary"
    - "Resilient"
    - "Innovative"
    - "Mentor"
potential: "With a rich background in engineering physics and hands-on experience in financial engineering, Kent possesses a unique ability to navigate complex systems and extract actionable insights. Their journey has been marked by a relentless pursuit of knowledge, mentorship of others, and a passion for leveraging technology to solve real-world problems. As they embark on this next chapter, challenges loom on 